# OLS Regression on Rental Data and Geospacial Data

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures, StandardScaler


Hej Philip.

Jeg har groft lavet interaktionsled, men det er lidt wack.

Tror ikke det giver mening at have interaktionsled mellem alt, som man gør i lasso. Vi har samme problem her som ved polinomials, da det på samme måde giver total meget overfitting. (vi får 200-300 led), derfor har vi snakket om, at det giver bedre mening bare at have interaktionsled mellem fx postkoderne og resten. ligesom vi snakkede om med TA'en. så vi inkludere interaktionsled på det som vi mener giver økonomisk mening og bygger modellen ud fra det. OLS kan nemlig ikke vurdere om det er godt at have flere med, den er bare altid glad for mere.

Jeg foreslår (og de andre var enige), at vi selv laver en model med interaktionsled, som man havde gjort i økonometri. 

Derved tænker jeg, at vi skal have interaktion mellem postkoder og de andre led. (måske ikke alle andre led, vi må lige vurdere om postkode og møbler fx gir mening)

Hvis du kan komme på andre som fx møbler og dyr eller sådan noget, så tilføj dem. men ellers tænker jeg bare postkoder.

Så må lassoen være den model der fitter de korrekte led, da vi ikke kan inkludere alle uden at total overfitte.

Hov vi kan selvfølgelig godt have en på elevator og floors, da man selvfølgelig hellere vil have elevator på 6. etage end 1. etage.

Andre ideer, baseret på deres p-værdi når de køres for alle. så disse kunne have betydning, men vi skal kun inkludere dem, hvis de gir mening rent praktisk/økonomisk:
Square meters_x_Rooms 
måske noget senior frendly og elevator
senier frendly og etage

Derudover kan det godt være at koden har nogle af interaktionsleddene med to gange. er ikke sikker men kunne måske være som: Square meters_x_Rooms og Rooms_x_Square meters


In [23]:
# Load the CSV file into a Pandas DataFrame
data = pd.read_csv('4_Final_Data.csv')

# Display the first 15 rows of the DataFrame
data.head(15)


,Latitude,Longitude,Monthly rent,Aconto,Square meters,Rooms,Furnished,Shareable,Pets allowed,Elevator,...,Floor_-1 to 0,Floor_1-3,Floor_3-8,Floor_9 or above,Distance to Transport Station (km),Distance to Beach (km),Distance to School (km),Distance to Restaurant (km),Distance to Hospital (km),Distance to Mall (km)
0,55.393856,10.394185,5400.0,800.0,62.0,2.0,0.0,0.0,0.0,1.0,...,0,1,0,0,0.227327,10.937984,0.347042,0.114848,1.856833,0.972492
1,56.300211,10.477949,7100.0,700.0,100.0,4.0,0.0,0.0,1.0,0.0,...,0,1,0,0,6.490199,5.187801,0.198759,0.081711,22.428327,20.441833
2,57.057390,9.936155,5800.0,300.0,64.0,2.0,0.0,0.0,0.0,0.0,...,0,1,0,0,2.057982,46.063274,0.894998,1.091345,1.726659,1.252278
3,55.721333,12.556324,8200.0,500.0,43.0,1.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0.684702,2.087369,0.121870,0.253656,0.873095,1.532133
4,56.199538,10.198303,8195.0,600.0,60.0,3.0,0.0,1.0,1.0,1.0,...,0,1,0,0,1.569096,12.856874,0.929082,0.441412,1.803299,0.677731
5,56.465990,10.034878,6300.0,650.0,109.0,3.0,0.0,0.0,1.0,0.0,...,0,1,0,0,0.875646,22.387664,0.123045,0.205792,0.278120,1.696457
6,55.973541,10.145876,7600.0,500.0,82.0,3.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0.352284,10.775682,0.403679,0.124096,16.340729,13.096304
7,55.465056,8.456541,8500.0,700.0,72.0,2.0,1.0,0.0,0.0,0.0,...,0,1,0,0,0.327681,0.887079,0.504533,0.089359,0.224610,0.138749
8,56.204861,9.015997,5995.0,0.0,94.0,3.0,0.0,0.0,0.0,1.0,...,0,1,0,0,8.899503,8.677817,0.285981,7.652507,8.303840,7.682953
9,55.243910,9.299112,5250.0,1250.0,85.0,3.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0.628434,19.025887,0.219876,0.215711,9.248625,31.633539


# Without Geospatial Data

In [19]:
# Prepare the features (X) and the label (y)
# Drop 'Monthly rent', 'Longitude' and 'Latitude' columns
X = data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0', 'Distance to Transport Station (km)', 'Distance to Beach (km)', 'Distance to School (km)', 'Distance to Restaurant (km)', 'Distance to Hospital (km)', 'Distance to Mall (km)'], axis=1)
y = data['Monthly rent']  # Labels (the 'Monthly rent' column)

# Add a constant to the features (intercept term)
X = sm.add_constant(X)

# Create an OLS model
model = sm.OLS(y, X)

# Fit the OLS model with heteroscedasticity robust errors
results = model.fit(cov_type='HC3')

# Print the summary of the regression
print(results.summary())

# Get the Predicted Values
y_pred = results.predict(X)

# Calculate the Residuals
residuals = y - y_pred

# Calculate the RMSE
RMSE = np.sqrt((residuals ** 2).mean())
print('Root Mean Squared Error (RMSE):', RMSE)

                            OLS Regression Results                            
Dep. Variable:           Monthly rent   R-squared:                       0.752
Model:                            OLS   Adj. R-squared:                  0.752
Method:                 Least Squares   F-statistic:                     737.7
Date:                Wed, 16 Aug 2023   Prob (F-statistic):               0.00
Time:                        15:47:07   Log-Likelihood:                -89813.
No. Observations:                9967   AIC:                         1.797e+05
Df Residuals:                    9944   BIC:                         1.798e+05
Df Model:                          22                                         
Covariance Type:                  HC3                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 1.253e+04 

# With Geospatial Data

In [20]:
# Prepare the features (X) and the label (y)
# Drop 'Monthly rent', 'Longitude' and 'Latitude' columns
X = data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0'], axis=1)
y = data['Monthly rent']  # Labels (the 'Monthly rent' column)

# Add a constant to the features (intercept term)
X = sm.add_constant(X)

# Create an OLS model
model = sm.OLS(y, X)

# Fit the OLS model with heteroscedasticity robust errors
results = model.fit(cov_type='HC3')

# Print the summary of the regression
print(results.summary())

# Get the Predicted Values
y_pred = results.predict(X)

# Calculate the Residuals
residuals = y - y_pred

# Calculate the RMSE
RMSE = np.sqrt((residuals ** 2).mean())
print('Root Mean Squared Error (RMSE):', RMSE)



                            OLS Regression Results                            
Dep. Variable:           Monthly rent   R-squared:                       0.778
Model:                            OLS   Adj. R-squared:                  0.778
Method:                 Least Squares   F-statistic:                     639.6
Date:                Wed, 16 Aug 2023   Prob (F-statistic):               0.00
Time:                        15:47:07   Log-Likelihood:                -89261.
No. Observations:                9967   AIC:                         1.786e+05
Df Residuals:                    9938   BIC:                         1.788e+05
Df Model:                          28                                         
Covariance Type:                  HC3                                         
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
cons

# Without Geospatial Data - All interactions

In [21]:
# Prepare the features (X) and the label (y)
# Drop the columns as before
X = data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0', 'Distance to Transport Station (km)', 'Distance to Beach (km)', 'Distance to School (km)', 'Distance to Restaurant (km)', 'Distance to Hospital (km)', 'Distance to Mall (km)'], axis=1)
y = data['Monthly rent']  # Labels (the 'Monthly rent' column)

# Generate interaction terms between all pairs of features and create names
interaction_terms = []
interaction_names = []
num_features = X.shape[1]

for i in range(num_features):
    for j in range(i + 1, num_features):
        feature1_name = X.columns[i]
        feature2_name = X.columns[j]
        
        # Exclude interactions within the same group
        if not (feature1_name.startswith('PostalCode') and feature2_name.startswith('PostalCode')) and not (feature1_name.startswith('Floor') and feature2_name.startswith('Floor')):
            interaction_term = X.iloc[:, i] * X.iloc[:, j]
            
            # Create an interaction term name
            name = f'{feature1_name}_x_{feature2_name}'
            interaction_names.append(name)
            
            interaction_terms.append(interaction_term)

# Combine the interaction terms into a new DataFrame
interaction_df = pd.concat(interaction_terms, axis=1)
interaction_df.columns = interaction_names

# Combine original features and interaction terms
X_with_interactions = pd.concat([X, interaction_df], axis=1)

# Add a constant to the features (intercept term)
X_with_interactions = sm.add_constant(X_with_interactions)

# Create an OLS model
model = sm.OLS(y, X_with_interactions)

# Fit the OLS model with heteroscedasticity robust errors
results = model.fit(cov_type='HC3')

# Print the summary of the regression
print(results.summary())

# Get the Predicted Values
y_pred = results.predict(X_with_interactions)

# Calculate the Residuals
residuals = y - y_pred

# Calculate the RMSE
RMSE = np.sqrt((residuals ** 2).mean())
print('Root Mean Squared Error (RMSE):', RMSE)

                            OLS Regression Results                            
Dep. Variable:           Monthly rent   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     166.1
Date:                Wed, 16 Aug 2023   Prob (F-statistic):               0.00
Time:                        15:47:09   Log-Likelihood:                -87712.
No. Observations:                9967   AIC:                         1.759e+05
Df Residuals:                    9747   BIC:                         1.774e+05
Df Model:                         219                                         
Covariance Type:                  HC3                                         
                                               coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

c:\Users\Bruger\anaconda3\lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 222, but rank is 218
  warnings.warn('covariance of constraints does not have full '


# With Geospatial Data - All interactions

In [22]:
# Prepare the features (X) and the label (y)
# Drop 'Monthly rent', 'Longitude' and 'Latitude' columns
X = data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0'], axis=1)
y = data['Monthly rent']  # Labels (the 'Monthly rent' column)

# Generate interaction terms between all pairs of features and create names
interaction_terms = []
interaction_names = []
num_features = X.shape[1]

for i in range(num_features):
    for j in range(i + 1, num_features):
        feature1_name = X.columns[i]
        feature2_name = X.columns[j]
        
        # Exclude interactions within the same group
        if not (feature1_name.startswith('PostalCode') and feature2_name.startswith('PostalCode')) and not (feature1_name.startswith('Floor') and feature2_name.startswith('Floor')):
            interaction_term = X.iloc[:, i] * X.iloc[:, j]
            
            # Create an interaction term name
            name = f'{feature1_name}_x_{feature2_name}'
            interaction_names.append(name)
            
            interaction_terms.append(interaction_term)

# Combine the interaction terms into a new DataFrame
interaction_df = pd.concat(interaction_terms, axis=1)
interaction_df.columns = interaction_names

# Combine original features and interaction terms
X_with_interactions = pd.concat([X, interaction_df], axis=1)

# Add a constant to the features (intercept term)
X_with_interactions = sm.add_constant(X_with_interactions)

# Create an OLS model
model = sm.OLS(y, X_with_interactions)

# Fit the OLS model with heteroscedasticity robust errors
results = model.fit(cov_type='HC3')

# Print the summary of the regression
print(results.summary())

# Get the Predicted Values
y_pred = results.predict(X_with_interactions)

# Calculate the Residuals
residuals = y - y_pred

# Calculate the RMSE
RMSE = np.sqrt((residuals ** 2).mean())
print('Root Mean Squared Error (RMSE):', RMSE)

c:\Users\Bruger\anaconda3\lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 375, but rank is 335
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:           Monthly rent   R-squared:                       0.891
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                     60.07
Date:                Wed, 16 Aug 2023   Prob (F-statistic):               0.00
Time:                        15:47:14   Log-Likelihood:                -85709.
No. Observations:                9967   AIC:                         1.722e+05
Df Residuals:                    9594   BIC:                         1.749e+05
Df Model:                         372                                         
Covariance Type:                  HC3                                         
                                                                       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------